# Metro map matching test Jupyter Notebook

Use this notebook to test shapes defined in `test_shape.txt` rather than running the entire `main.py` script.

Outputs results to a local file `test_shape.html`. Blue lines denote the existing shape, and red lines are the map matched shape.

In [8]:
import os
import requests
import json
import folium
import polyline
import pandas as pd
from dotenv import load_dotenv

load_dotenv()

True

## Plot existing shapes

In [9]:
test_shape_df = pd.read_csv('test_shape.txt')
grouped_shapes = test_shape_df.groupby('shape_id')

m = folium.Map(
    location=[test_shape_df['shape_pt_lat'].mean(), test_shape_df['shape_pt_lon'].mean()], 
    zoom_start=12
)

for shape_id, group in grouped_shapes:
    points = list(zip(group['shape_pt_lat'], group['shape_pt_lon']))
    folium.PolyLine(points, color="blue", weight=2.5, opacity=1).add_to(m)

## Perform map matching API call and add result to map

In [ ]:
trace_attribute_headers = {
    'Content-Type': 'application/json',
    'Cache-Control': 'no-cache'
}

for shape_id, group in grouped_shapes:
    points = []
    for idx, row in group.iterrows():
        point_type = "via"
        if idx == group.index[0]:
            point_type = "break"
        elif idx == group.index[-1]:
            point_type = "break"
        points.append({
            "lat": float(row["shape_pt_lat"]),
            "lon": float(row["shape_pt_lon"]),
            "type": point_type
        })

    trace_attribute_json = {
        "shape": points,
        "costing": "bus",
        "costing_options": {
            "bus": {
                "ignore_oneways": True,
                "ignore_restrictions": True,
                "ignore_access": True,
                "private_access_penalty": 0
            }
        },
        "shape_match": "map_snap",
    }

    trace_attribute_response = requests.post(
        url=os.getenv('VALHALLA_URL')+'/trace_attributes',
        headers=trace_attribute_headers,
        data=json.dumps(trace_attribute_json)
    )

    matched_path = json.loads(trace_attribute_response.content)
    shape_points = polyline.decode(matched_path["shape"], precision=6)
    folium.PolyLine(shape_points, color="red", weight=2.5, opacity=1).add_to(m)

    m.save('test_shape.html')

    # Save each shape as a separate JSON file
    #with open(f'shape_{shape_id}.json', "w") as f:
    #    json.dump(trace_route_json, f, indent=2)